In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import pyplot
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from keras.models import Sequential
from keras.layers import Dense,InputLayer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR

In [2]:
filePath='/Users/a0p07yx/Documents/Projects/DellDS/Temp/Dataset.xlsx'

In [3]:
df_train = pd.read_excel(filePath, sheet_name="Train set",engine='openpyxl')
df_valid = pd.read_excel(filePath, sheet_name="Validation",engine='openpyxl')
df_test = pd.read_excel(filePath, sheet_name="Test Set",engine='openpyxl')
df_dict = pd.read_excel(filePath, sheet_name="Dictionary",skiprows=2,engine='openpyxl')

In [4]:
df_dict=df_dict[['CPU','RAM','Storage','GPU']]
CPU_list=df_dict['CPU'].dropna().values
print('CPU',len(CPU_list))
RAM_list=df_dict['RAM'].dropna().values
print('RAM',len(RAM_list))
Storage_list=df_dict['Storage'].dropna().values
print('Storage',len(Storage_list))
GPU_list=df_dict['GPU'].dropna().values
print('GPU',len(GPU_list))

CPU 44
RAM 17
Storage 76
GPU 36


In [5]:
def create_ranking_dict(temp_list):
    #creating rank
    temp_dict={}
    for rank,val in enumerate(temp_list):
        temp_dict[val]=int(rank)+1
    return temp_dict 

In [6]:
#creating rank
CPU_dict=create_ranking_dict(CPU_list)
RAM_dict=create_ranking_dict(RAM_list)
Storage_dict=create_ranking_dict(Storage_list)
GPU_dict=create_ranking_dict(GPU_list)

In [7]:
df_train=df_train.dropna()
df_train['ScreenSize']=df_train['ScreenSize'].astype(str)
df_train['ScreenType']=df_train['ScreenType'].astype(str)
df_train['TouchType']=df_train['TouchType'].astype(str)
df_train['Form']=df_train['Form'].astype(str)
df_train['Region']=df_train['Region'].astype(str)
df_train.shape

(3159, 10)

In [8]:
df_train[df_train['Form'].isna()==True]

,CPU,RAM,Storage,GPU,ScreenSize,ScreenType,TouchType,Form,Region,Price in $


In [9]:
### training data

In [10]:
#### maping rank to dataframe
df_rank=df_train
df_rank['CPU']=df_train['CPU'].map(CPU_dict)
df_rank['RAM']=df_train['RAM'].map(RAM_dict)
df_rank['Storage']=df_train['Storage'].map(Storage_dict)
df_rank['GPU']=df_train['GPU'].map(GPU_dict)
df_rank.shape

(3159, 10)

In [11]:
df_rank.head()

,CPU,RAM,Storage,GPU,ScreenSize,ScreenType,TouchType,Form,Region,Price in $
0,19,10,10,1,15,UHD,Non-Touch,Laptop,CN,2803.20
1,19,10,10,1,14,FHD,Non-Touch,Laptop,CN,719.85
2,19,10,10,1,15,FHD,Touch,Laptop,BR,1584.00
3,19,10,10,1,13,FHD,Touch,Laptop,BR,1872.00
4,19,10,10,1,15,FHD,Non-Touch,Laptop,BR,1782.00


In [12]:
### training data

In [13]:
X_train=df_rank.drop(['Price in $'],axis=1)
y_train=df_rank['Price in $']

In [14]:
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import (ElasticNetCV, LassoCV, RidgeCV, LinearRegression)
from catboost import CatBoostRegressor
from sklearn.ensemble import (RandomForestRegressor, StackingRegressor)
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import cross_val_score

In [15]:
### creating pipeline
num_pipeline = Pipeline([('std_scaler', StandardScaler())])
cat_pipeline = Pipeline([('cat', OneHotEncoder(sparse=False,handle_unknown='ignore'))])
neutral_attribs = ['CPU','RAM','Storage','GPU']
cat_attribs = ['ScreenSize','ScreenType','TouchType','Form','Region']
#neutral_attribs=[]

full_pipeline = ColumnTransformer(transformers=[
       ("dont_tranforst","passthrough",neutral_attribs),
        #("num", num_pipeline, num_attribs),
        ("cat", cat_pipeline, cat_attribs),
    ],remainder='drop')


In [16]:
def get_mape(pipeline, X, y, folds):

    mape = -1*cross_val_score(pipeline, X, y, cv=folds, scoring='neg_mean_absolute_percentage_error')
    
    return mape

In [18]:
models = [
    XGBRegressor(),
    RandomForestRegressor(),
    LGBMRegressor()
]

#  Define estimators to stacked regressor
estimator_names = ['XGBRegressor','random_forest', 'lgbmregressor', ]
estimators = [(estimator_names[i], model) for i, model in enumerate(models)]
    
#  Define stacked model with Linear Regression as the final estimator
Stacked = StackingRegressor(estimators=estimators, final_estimator=CatBoostRegressor(), cv=5)

#  Add stacked model to models list from previous step
models.append(Stacked)


MAPE = []
model_names = []
for i, model in enumerate(models):
    
    model_pipeline = Pipeline(steps=[('pipeline', full_pipeline),
                                    ('model', model)])

    #  Fit training data and score
    
    MAPE.append(get_mape(model_pipeline, X_train, y_train, 5))
    
    #  Print the scores
    
    model_names.append(str(model).split('(')[0])
    print('{} Training Score: {}'.format(model_names[i], round(np.mean(MAPE[i]),4)))
    
    

XGBRegressor Training Score: 0.2535
RandomForestRegressor Training Score: 0.2664
LGBMRegressor Training Score: 0.2539
Learning rate set to 0.046032
0:	learn: 1181.2202043	total: 55.2ms	remaining: 55.2s
1:	learn: 1143.9265116	total: 56.5ms	remaining: 28.2s
2:	learn: 1108.7607831	total: 57.8ms	remaining: 19.2s
3:	learn: 1074.4741308	total: 58.9ms	remaining: 14.7s
4:	learn: 1042.8629176	total: 60ms	remaining: 11.9s
5:	learn: 1012.0200696	total: 61.3ms	remaining: 10.2s
6:	learn: 982.4940721	total: 63ms	remaining: 8.94s
7:	learn: 955.0099785	total: 65.7ms	remaining: 8.14s
8:	learn: 928.6982942	total: 68ms	remaining: 7.49s
9:	learn: 905.8410760	total: 70.6ms	remaining: 6.99s
10:	learn: 882.6914896	total: 72.9ms	remaining: 6.55s
11:	learn: 860.3801377	total: 74.9ms	remaining: 6.17s
12:	learn: 839.1108753	total: 76.8ms	remaining: 5.83s
13:	learn: 819.2851489	total: 78.7ms	remaining: 5.54s
14:	learn: 800.7432092	total: 80.3ms	remaining: 5.27s
15:	learn: 783.5653368	total: 82ms	remaining: 5.04s


210:	learn: 511.0114144	total: 339ms	remaining: 1.27s
211:	learn: 510.7544280	total: 341ms	remaining: 1.27s
212:	learn: 510.6865464	total: 342ms	remaining: 1.26s
213:	learn: 510.6220947	total: 343ms	remaining: 1.26s
214:	learn: 510.5304262	total: 345ms	remaining: 1.26s
215:	learn: 509.9931895	total: 346ms	remaining: 1.25s
216:	learn: 509.9244283	total: 347ms	remaining: 1.25s
217:	learn: 509.8624947	total: 348ms	remaining: 1.25s
218:	learn: 509.7967339	total: 350ms	remaining: 1.25s
219:	learn: 509.7069562	total: 351ms	remaining: 1.24s
220:	learn: 509.5983146	total: 352ms	remaining: 1.24s
221:	learn: 509.5052953	total: 353ms	remaining: 1.24s
222:	learn: 509.4495934	total: 354ms	remaining: 1.23s
223:	learn: 509.3904450	total: 355ms	remaining: 1.23s
224:	learn: 509.3389821	total: 357ms	remaining: 1.23s
225:	learn: 509.2867166	total: 358ms	remaining: 1.23s
226:	learn: 509.2406695	total: 359ms	remaining: 1.22s
227:	learn: 509.1963875	total: 360ms	remaining: 1.22s
228:	learn: 509.1327475	tota

491:	learn: 450.5720100	total: 680ms	remaining: 702ms
492:	learn: 450.5455965	total: 682ms	remaining: 701ms
493:	learn: 450.3652780	total: 683ms	remaining: 699ms
494:	learn: 450.0349879	total: 684ms	remaining: 698ms
495:	learn: 449.8443733	total: 685ms	remaining: 696ms
496:	learn: 449.6639055	total: 686ms	remaining: 694ms
497:	learn: 449.4496526	total: 687ms	remaining: 693ms
498:	learn: 449.2723740	total: 688ms	remaining: 691ms
499:	learn: 449.0586813	total: 690ms	remaining: 690ms
500:	learn: 448.8392308	total: 691ms	remaining: 688ms
501:	learn: 448.5862242	total: 692ms	remaining: 686ms
502:	learn: 448.3884899	total: 693ms	remaining: 685ms
503:	learn: 448.2724592	total: 694ms	remaining: 683ms
504:	learn: 448.0778816	total: 696ms	remaining: 682ms
505:	learn: 447.8241576	total: 697ms	remaining: 680ms
506:	learn: 447.7649931	total: 698ms	remaining: 679ms
507:	learn: 447.5267879	total: 699ms	remaining: 677ms
508:	learn: 447.3023940	total: 700ms	remaining: 675ms
509:	learn: 447.1452152	tota

779:	learn: 406.1219870	total: 1.03s	remaining: 291ms
780:	learn: 405.8659725	total: 1.03s	remaining: 290ms
781:	learn: 405.6769472	total: 1.03s	remaining: 288ms
782:	learn: 405.6629466	total: 1.03s	remaining: 287ms
783:	learn: 405.6081454	total: 1.04s	remaining: 286ms
784:	learn: 405.5485238	total: 1.04s	remaining: 284ms
785:	learn: 405.4110722	total: 1.04s	remaining: 283ms
786:	learn: 405.3162256	total: 1.04s	remaining: 282ms
787:	learn: 405.2367334	total: 1.04s	remaining: 280ms
788:	learn: 405.0858410	total: 1.04s	remaining: 279ms
789:	learn: 404.9888043	total: 1.04s	remaining: 278ms
790:	learn: 404.8490743	total: 1.04s	remaining: 276ms
791:	learn: 404.7059873	total: 1.05s	remaining: 275ms
792:	learn: 404.6221903	total: 1.05s	remaining: 274ms
793:	learn: 404.4577496	total: 1.05s	remaining: 272ms
794:	learn: 404.3856190	total: 1.05s	remaining: 271ms
795:	learn: 404.3070626	total: 1.05s	remaining: 270ms
796:	learn: 404.1477627	total: 1.05s	remaining: 268ms
797:	learn: 404.0248720	tota

Learning rate set to 0.046032
0:	learn: 1109.5458400	total: 1.74ms	remaining: 1.74s
1:	learn: 1075.8463700	total: 3.43ms	remaining: 1.71s
2:	learn: 1044.0494306	total: 5.04ms	remaining: 1.68s
3:	learn: 1014.5355555	total: 6.94ms	remaining: 1.73s
4:	learn: 986.1574059	total: 8.37ms	remaining: 1.66s
5:	learn: 959.3821300	total: 10.1ms	remaining: 1.67s
6:	learn: 933.6395861	total: 11.7ms	remaining: 1.66s
7:	learn: 910.5438757	total: 13.4ms	remaining: 1.66s
8:	learn: 887.7829964	total: 15ms	remaining: 1.65s
9:	learn: 867.2195509	total: 16.4ms	remaining: 1.62s
10:	learn: 846.5103935	total: 18.4ms	remaining: 1.65s
11:	learn: 826.7644025	total: 20.1ms	remaining: 1.65s
12:	learn: 809.0394616	total: 22.1ms	remaining: 1.68s
13:	learn: 791.9353522	total: 23.6ms	remaining: 1.66s
14:	learn: 775.5877550	total: 24.9ms	remaining: 1.63s
15:	learn: 760.2831031	total: 26.2ms	remaining: 1.61s
16:	learn: 745.9380349	total: 27.5ms	remaining: 1.59s
17:	learn: 732.5908295	total: 28.7ms	remaining: 1.57s
18:	le

...

133:	learn: 440.0161846	total: 171ms	remaining: 1.11s
134:	learn: 439.8864630	total: 173ms	remaining: 1.11s
135:	learn: 439.7780775	total: 174ms	remaining: 1.1s
136:	learn: 439.6374497	total: 175ms	remaining: 1.1s
137:	learn: 439.2011846	total: 176ms	remaining: 1.1s
138:	learn: 438.9350792	total: 177ms	remaining: 1.1s
139:	learn: 438.7472157	total: 178ms	remaining: 1.09s
140:	learn: 438.6507951	total: 179ms	remaining: 1.09s
141:	learn: 438.4369181	total: 180ms	remaining: 1.09s
142:	learn: 438.3248755	total: 182ms	remaining: 1.09s
143:	learn: 438.1874596	total: 183ms	remaining: 1.09s
144:	learn: 437.9178559	total: 184ms	remaining: 1.09s
145:	learn: 437.8054799	total: 186ms	remaining: 1.09s
146:	learn: 437.7703462	total: 187ms	remaining: 1.09s
147:	learn: 437.7421697	total: 189ms	remaining: 1.09s
148:	learn: 437.6859702	total: 190ms	remaining: 1.08s
149:	learn: 437.5772260	total: 191ms	remaining: 1.08s
150:	learn: 437.2646831	total: 192ms	remaining: 1.08s
151:	learn: 437.2366970	total: 1

364:	learn: 394.7317930	total: 514ms	remaining: 895ms
365:	learn: 394.5396861	total: 516ms	remaining: 894ms
366:	learn: 394.3851527	total: 518ms	remaining: 893ms
367:	learn: 394.2583133	total: 520ms	remaining: 893ms
368:	learn: 393.9744156	total: 522ms	remaining: 892ms
369:	learn: 393.8719070	total: 523ms	remaining: 891ms
370:	learn: 393.6976587	total: 525ms	remaining: 890ms
371:	learn: 393.5228549	total: 526ms	remaining: 889ms
372:	learn: 393.3806949	total: 528ms	remaining: 887ms
373:	learn: 393.1140061	total: 529ms	remaining: 886ms
374:	learn: 392.9679793	total: 531ms	remaining: 885ms
375:	learn: 392.9014846	total: 533ms	remaining: 884ms
376:	learn: 392.7692222	total: 534ms	remaining: 882ms
377:	learn: 392.6226277	total: 535ms	remaining: 880ms
378:	learn: 392.5381668	total: 536ms	remaining: 878ms
379:	learn: 392.3903965	total: 537ms	remaining: 876ms
380:	learn: 392.1177028	total: 538ms	remaining: 874ms
381:	learn: 391.8737236	total: 539ms	remaining: 873ms
382:	learn: 391.6749567	tota

551:	learn: 369.6217369	total: 861ms	remaining: 699ms
552:	learn: 369.5647480	total: 863ms	remaining: 698ms
553:	learn: 369.3803559	total: 864ms	remaining: 696ms
554:	learn: 369.1191392	total: 866ms	remaining: 695ms
555:	learn: 368.9871350	total: 868ms	remaining: 693ms
556:	learn: 368.9262430	total: 870ms	remaining: 692ms
557:	learn: 368.8144013	total: 871ms	remaining: 690ms
558:	learn: 368.7843976	total: 873ms	remaining: 688ms
559:	learn: 368.7065304	total: 874ms	remaining: 687ms
560:	learn: 368.5721611	total: 876ms	remaining: 685ms
561:	learn: 368.5025488	total: 877ms	remaining: 683ms
562:	learn: 368.3538554	total: 878ms	remaining: 682ms
563:	learn: 368.2855218	total: 879ms	remaining: 680ms
564:	learn: 368.2188034	total: 881ms	remaining: 678ms
565:	learn: 368.1451799	total: 882ms	remaining: 676ms
566:	learn: 368.0794793	total: 883ms	remaining: 674ms
567:	learn: 367.9823027	total: 884ms	remaining: 673ms
568:	learn: 367.8413201	total: 887ms	remaining: 672ms
569:	learn: 367.6806403	tota

796:	learn: 349.2930017	total: 1.2s	remaining: 306ms
797:	learn: 349.2127684	total: 1.2s	remaining: 305ms
798:	learn: 349.0988159	total: 1.21s	remaining: 303ms
799:	learn: 348.9513047	total: 1.21s	remaining: 302ms
800:	learn: 348.8863390	total: 1.21s	remaining: 300ms
801:	learn: 348.7668179	total: 1.21s	remaining: 299ms
802:	learn: 348.7323607	total: 1.21s	remaining: 297ms
803:	learn: 348.6646543	total: 1.21s	remaining: 295ms
804:	learn: 348.6187316	total: 1.21s	remaining: 294ms
805:	learn: 348.4005467	total: 1.21s	remaining: 292ms
806:	learn: 348.3470413	total: 1.22s	remaining: 291ms
807:	learn: 348.2762362	total: 1.22s	remaining: 289ms
808:	learn: 348.1153190	total: 1.22s	remaining: 288ms
809:	learn: 348.1077660	total: 1.22s	remaining: 286ms
810:	learn: 347.9948122	total: 1.22s	remaining: 285ms
811:	learn: 347.9300951	total: 1.22s	remaining: 283ms
812:	learn: 347.8958842	total: 1.23s	remaining: 282ms
813:	learn: 347.8496051	total: 1.23s	remaining: 280ms
814:	learn: 347.7160417	total:

StackingRegressor Training Score: 0.2561


In [19]:
models 

[XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=None, tree_method=None,
              validate_parameters=None, verbosity=None),
 RandomForestRegressor(),
 LGBMRegressor(),
 StackingRegressor(cv=5,
                   estimators=[('XGBRegressor',
                                XGBRegressor(base_score=None, booster=None,
                                             colsample_bylevel=None,
                                             colsample_bynode=None,
                                     

In [20]:
models = [
    SVR(),
    CatBoostRegressor(),
    RandomForestRegressor(),
    ElasticNetCV(),
    LassoCV(), 
    RidgeCV(), 
    LGBMRegressor()
]

In [21]:
model_pipeline = Pipeline(steps=[('pipeline', full_pipeline),
                                    ('model', models[0])])
print(models[0],get_mape(model_pipeline, X_train, y_train, 5))

SVR() [0.5014862  0.55895003 0.61511766 0.38995123 0.51001368]


In [22]:
model_pipeline = Pipeline(steps=[('pipeline', full_pipeline),
                                    ('model', models[2])])
print(models[2],get_mape(model_pipeline, X_train, y_train, 5))

RandomForestRegressor() [0.22618544 0.26146559 0.2733309  0.23614755 0.32708953]


In [23]:
model_pipeline = Pipeline(steps=[('pipeline', full_pipeline),
                                    ('model', models[2])])
print(models[3],get_mape(model_pipeline, X_train, y_train, 3))

ElasticNetCV() [0.23703721 0.31375314 0.23520594]


In [24]:
model_pipeline = Pipeline(steps=[('pipeline', full_pipeline),
                                    ('model', models[4])])
print(models[4],get_mape(model_pipeline, X_train, y_train, 5))

LassoCV() [0.44242063 0.51259823 0.54376719 0.48349712 0.3951114 ]


In [25]:
model_pipeline = Pipeline(steps=[('pipeline', full_pipeline),
                                    ('model', models[5])])
print(models[5],get_mape(model_pipeline, X_train, y_train, 5))

RidgeCV(alphas=array([ 0.1,  1. , 10. ])) [0.44658448 0.5043375  0.54944529 0.47881733 0.39898357]


In [26]:
model_pipeline = Pipeline(steps=[('pipeline', full_pipeline),
                                    ('model', models[6])])
print(models[6],get_mape(model_pipeline, X_train, y_train, 5))

LGBMRegressor() [0.24317159 0.25981114 0.25377583 0.22267672 0.28982453]


In [27]:
model_pipeline = Pipeline(steps=[('pipeline', full_pipeline),
                                    ('model', XGBRegressor())])
print('XGBRegressor()',get_mape(model_pipeline, X_train, y_train, 5))

XGBRegressor() [0.23922438 0.2328161  0.24244994 0.21997733 0.33285716]


In [28]:
model_pipeline = Pipeline(steps=[('pipeline', full_pipeline),
                                    ('model', CatBoostRegressor())])
print('catRegressor()',get_mape(model_pipeline, X_train, y_train, 5))

Learning rate set to 0.046032
0:	learn: 1189.8344179	total: 2.02ms	remaining: 2.02s
1:	learn: 1162.7246659	total: 3.37ms	remaining: 1.68s
2:	learn: 1135.6010309	total: 5.02ms	remaining: 1.67s
3:	learn: 1110.4704781	total: 6.2ms	remaining: 1.54s
4:	learn: 1084.7381570	total: 7.67ms	remaining: 1.53s
5:	learn: 1060.8280467	total: 9.14ms	remaining: 1.51s
6:	learn: 1038.3623748	total: 10.9ms	remaining: 1.54s
7:	learn: 1017.9543647	total: 12.5ms	remaining: 1.55s
8:	learn: 997.3294809	total: 14.3ms	remaining: 1.58s
9:	learn: 976.8608123	total: 15.9ms	remaining: 1.58s
10:	learn: 961.1275344	total: 17.4ms	remaining: 1.56s
11:	learn: 945.5350559	total: 18.9ms	remaining: 1.56s
12:	learn: 928.7133529	total: 20.4ms	remaining: 1.55s
13:	learn: 914.1011251	total: 22.2ms	remaining: 1.56s
14:	learn: 898.9552100	total: 23.9ms	remaining: 1.57s
15:	learn: 885.0764021	total: 25.4ms	remaining: 1.56s
16:	learn: 870.4393757	total: 27.2ms	remaining: 1.57s
17:	learn: 858.8540626	total: 28.6ms	remaining: 1.56s
1

197:	learn: 426.3053918	total: 270ms	remaining: 1.09s
198:	learn: 425.7192833	total: 272ms	remaining: 1.1s
199:	learn: 425.0668588	total: 274ms	remaining: 1.1s
200:	learn: 424.6058512	total: 276ms	remaining: 1.1s
201:	learn: 424.0947243	total: 277ms	remaining: 1.09s
202:	learn: 423.3430910	total: 279ms	remaining: 1.09s
203:	learn: 422.6492080	total: 280ms	remaining: 1.09s
204:	learn: 422.1763471	total: 282ms	remaining: 1.09s
205:	learn: 421.3268603	total: 283ms	remaining: 1.09s
206:	learn: 420.5342043	total: 284ms	remaining: 1.09s
207:	learn: 419.9673915	total: 286ms	remaining: 1.09s
208:	learn: 419.2976366	total: 288ms	remaining: 1.09s
209:	learn: 418.2529597	total: 289ms	remaining: 1.09s
210:	learn: 417.5621934	total: 290ms	remaining: 1.08s
211:	learn: 416.5946795	total: 292ms	remaining: 1.08s
212:	learn: 415.9118686	total: 293ms	remaining: 1.08s
213:	learn: 415.2489692	total: 294ms	remaining: 1.08s
214:	learn: 414.7475293	total: 295ms	remaining: 1.08s
215:	learn: 414.1012864	total: 

369:	learn: 348.1246666	total: 514ms	remaining: 876ms
370:	learn: 347.8515860	total: 516ms	remaining: 875ms
371:	learn: 347.6626306	total: 517ms	remaining: 873ms
372:	learn: 347.4985582	total: 519ms	remaining: 872ms
373:	learn: 347.1897708	total: 520ms	remaining: 871ms
374:	learn: 346.9254479	total: 522ms	remaining: 869ms
375:	learn: 346.5683025	total: 523ms	remaining: 868ms
376:	learn: 346.3551225	total: 525ms	remaining: 867ms
377:	learn: 346.0274372	total: 526ms	remaining: 866ms
378:	learn: 345.8079834	total: 528ms	remaining: 865ms
379:	learn: 345.4758678	total: 529ms	remaining: 864ms
380:	learn: 345.1566403	total: 531ms	remaining: 862ms
381:	learn: 344.9040139	total: 532ms	remaining: 861ms
382:	learn: 344.6227864	total: 534ms	remaining: 860ms
383:	learn: 344.4714897	total: 535ms	remaining: 858ms
384:	learn: 344.2264053	total: 537ms	remaining: 858ms
385:	learn: 343.9016499	total: 538ms	remaining: 856ms
386:	learn: 343.6165839	total: 540ms	remaining: 856ms
387:	learn: 343.4565291	tota

597:	learn: 305.4068261	total: 799ms	remaining: 537ms
598:	learn: 305.0586314	total: 800ms	remaining: 536ms
599:	learn: 304.9694377	total: 801ms	remaining: 534ms
600:	learn: 304.7673125	total: 803ms	remaining: 533ms
601:	learn: 304.6531798	total: 804ms	remaining: 531ms
602:	learn: 304.5145057	total: 805ms	remaining: 530ms
603:	learn: 304.4070536	total: 806ms	remaining: 528ms
604:	learn: 304.2255751	total: 807ms	remaining: 527ms
605:	learn: 304.0228115	total: 808ms	remaining: 526ms
606:	learn: 303.8855615	total: 810ms	remaining: 524ms
607:	learn: 303.7769057	total: 811ms	remaining: 523ms
608:	learn: 303.5026602	total: 813ms	remaining: 522ms
609:	learn: 303.3966732	total: 814ms	remaining: 520ms
610:	learn: 303.2500950	total: 815ms	remaining: 519ms
611:	learn: 303.1707336	total: 816ms	remaining: 518ms
612:	learn: 303.0716242	total: 818ms	remaining: 516ms
613:	learn: 303.0618510	total: 819ms	remaining: 515ms
614:	learn: 302.9674414	total: 820ms	remaining: 513ms
615:	learn: 302.8708477	tota

771:	learn: 282.0177529	total: 1.04s	remaining: 306ms
772:	learn: 281.8696242	total: 1.04s	remaining: 305ms
773:	learn: 281.7586379	total: 1.04s	remaining: 304ms
774:	learn: 281.6205087	total: 1.04s	remaining: 303ms
775:	learn: 281.4129361	total: 1.04s	remaining: 302ms
776:	learn: 281.2962718	total: 1.05s	remaining: 300ms
777:	learn: 281.1907728	total: 1.05s	remaining: 299ms
778:	learn: 281.0855576	total: 1.05s	remaining: 298ms
779:	learn: 280.9361595	total: 1.05s	remaining: 296ms
780:	learn: 280.8825356	total: 1.05s	remaining: 295ms
781:	learn: 280.7493414	total: 1.05s	remaining: 293ms
782:	learn: 280.5730113	total: 1.05s	remaining: 292ms
783:	learn: 280.4179092	total: 1.05s	remaining: 291ms
784:	learn: 280.3031830	total: 1.06s	remaining: 290ms
785:	learn: 280.2966497	total: 1.06s	remaining: 288ms
786:	learn: 280.2738959	total: 1.06s	remaining: 287ms
787:	learn: 280.2692169	total: 1.06s	remaining: 286ms
788:	learn: 280.1618167	total: 1.06s	remaining: 284ms
789:	learn: 280.1242269	tota

...

172:	learn: 424.1981820	total: 179ms	remaining: 858ms
173:	learn: 423.6860217	total: 181ms	remaining: 858ms
174:	learn: 422.8914017	total: 182ms	remaining: 860ms
175:	learn: 422.1338805	total: 184ms	remaining: 859ms
176:	learn: 421.1001017	total: 185ms	remaining: 859ms
177:	learn: 420.7523654	total: 186ms	remaining: 857ms
178:	learn: 420.1473800	total: 187ms	remaining: 856ms
179:	learn: 419.5030647	total: 188ms	remaining: 855ms
180:	learn: 418.6099610	total: 189ms	remaining: 854ms
181:	learn: 417.6710477	total: 190ms	remaining: 853ms
182:	learn: 416.6223601	total: 191ms	remaining: 852ms
183:	learn: 415.6751334	total: 192ms	remaining: 851ms
184:	learn: 414.9673608	total: 193ms	remaining: 850ms
185:	learn: 414.0026189	total: 194ms	remaining: 849ms
186:	learn: 413.3747885	total: 195ms	remaining: 848ms
187:	learn: 412.9329917	total: 196ms	remaining: 847ms
188:	learn: 412.2496048	total: 197ms	remaining: 846ms
189:	learn: 411.7349978	total: 198ms	remaining: 845ms
190:	learn: 411.2312196	tota

337:	learn: 341.8876435	total: 356ms	remaining: 697ms
338:	learn: 341.6098630	total: 357ms	remaining: 697ms
339:	learn: 341.3698273	total: 359ms	remaining: 696ms
340:	learn: 341.0693118	total: 360ms	remaining: 695ms
341:	learn: 340.6844815	total: 361ms	remaining: 694ms
342:	learn: 340.5162173	total: 362ms	remaining: 693ms
343:	learn: 340.2880324	total: 363ms	remaining: 692ms
344:	learn: 339.9899104	total: 364ms	remaining: 692ms
345:	learn: 339.7080934	total: 365ms	remaining: 691ms
346:	learn: 339.5026258	total: 366ms	remaining: 689ms
347:	learn: 339.4425425	total: 367ms	remaining: 688ms
348:	learn: 339.1580255	total: 368ms	remaining: 687ms
349:	learn: 339.0661405	total: 369ms	remaining: 686ms
350:	learn: 339.0108497	total: 370ms	remaining: 685ms
351:	learn: 338.7005779	total: 371ms	remaining: 684ms
352:	learn: 338.5593577	total: 373ms	remaining: 683ms
353:	learn: 338.2144118	total: 374ms	remaining: 682ms
354:	learn: 338.0036180	total: 375ms	remaining: 681ms
355:	learn: 337.5898755	tota

493:	learn: 307.4072440	total: 530ms	remaining: 543ms
494:	learn: 307.2307722	total: 532ms	remaining: 543ms
495:	learn: 307.1692694	total: 533ms	remaining: 542ms
496:	learn: 307.0596598	total: 534ms	remaining: 541ms
497:	learn: 306.8791154	total: 535ms	remaining: 540ms
498:	learn: 306.7713299	total: 536ms	remaining: 539ms
499:	learn: 306.5378069	total: 538ms	remaining: 538ms
500:	learn: 306.4184083	total: 539ms	remaining: 537ms
501:	learn: 306.2972579	total: 540ms	remaining: 536ms
502:	learn: 306.1931088	total: 541ms	remaining: 534ms
503:	learn: 305.9710229	total: 542ms	remaining: 533ms
504:	learn: 305.8967227	total: 543ms	remaining: 532ms
505:	learn: 305.6678183	total: 544ms	remaining: 531ms
506:	learn: 305.6271247	total: 545ms	remaining: 530ms
507:	learn: 305.4523982	total: 547ms	remaining: 529ms
508:	learn: 305.0525382	total: 548ms	remaining: 528ms
509:	learn: 304.8363010	total: 549ms	remaining: 527ms
510:	learn: 304.6716009	total: 550ms	remaining: 526ms
511:	learn: 304.6226020	tota

791:	learn: 267.0267751	total: 883ms	remaining: 232ms
792:	learn: 267.0022461	total: 885ms	remaining: 231ms
793:	learn: 266.8790535	total: 886ms	remaining: 230ms
794:	learn: 266.7007080	total: 887ms	remaining: 229ms
795:	learn: 266.6255586	total: 888ms	remaining: 228ms
796:	learn: 266.4892701	total: 890ms	remaining: 227ms
797:	learn: 266.3695690	total: 891ms	remaining: 225ms
798:	learn: 266.3245180	total: 892ms	remaining: 224ms
799:	learn: 266.1804523	total: 893ms	remaining: 223ms
800:	learn: 266.0380294	total: 895ms	remaining: 222ms
801:	learn: 265.9101584	total: 896ms	remaining: 221ms
802:	learn: 265.8344517	total: 897ms	remaining: 220ms
803:	learn: 265.6560207	total: 898ms	remaining: 219ms
804:	learn: 265.5408422	total: 899ms	remaining: 218ms
805:	learn: 265.4346924	total: 900ms	remaining: 217ms
806:	learn: 265.3487685	total: 901ms	remaining: 215ms
807:	learn: 265.1701072	total: 902ms	remaining: 214ms
808:	learn: 265.1010704	total: 903ms	remaining: 213ms
809:	learn: 265.0223178	tota

949:	learn: 253.3266341	total: 1.06s	remaining: 55.7ms
950:	learn: 253.2406096	total: 1.06s	remaining: 54.7ms
951:	learn: 253.1098009	total: 1.06s	remaining: 53.6ms
952:	learn: 253.0385581	total: 1.06s	remaining: 52.4ms
953:	learn: 252.9598728	total: 1.06s	remaining: 51.3ms
954:	learn: 252.8847816	total: 1.06s	remaining: 50.2ms
955:	learn: 252.8042761	total: 1.07s	remaining: 49.1ms
956:	learn: 252.6831934	total: 1.07s	remaining: 48ms
957:	learn: 252.6350213	total: 1.07s	remaining: 46.9ms
958:	learn: 252.4164677	total: 1.07s	remaining: 45.8ms
959:	learn: 252.2941042	total: 1.07s	remaining: 44.7ms
960:	learn: 252.2111097	total: 1.07s	remaining: 43.5ms
961:	learn: 252.1106617	total: 1.07s	remaining: 42.4ms
962:	learn: 252.0338552	total: 1.07s	remaining: 41.3ms
963:	learn: 251.8999559	total: 1.08s	remaining: 40.2ms
964:	learn: 251.7343828	total: 1.08s	remaining: 39.1ms
965:	learn: 251.7069667	total: 1.08s	remaining: 38ms
966:	learn: 251.6249829	total: 1.08s	remaining: 36.9ms
967:	learn: 25

In [29]:
### selected models catboost, randomforest, xgboost,lgbm

In [30]:
### validation data

In [31]:
#### maping rank to dataframe
df_valid.replace({'RAM 10S 2': 'RAM 10', 'RAM 14S 3': 'RAM 14' },inplace=True)
df_valid=df_valid
df_valid['CPU']=df_valid['CPU'].map(CPU_dict)
df_valid['RAM']=df_valid['RAM'].map(RAM_dict)
df_valid['Storage']=df_valid['Storage'].map(Storage_dict)
df_valid['GPU']=df_valid['GPU'].map(GPU_dict)
df_valid.shape

(754, 10)

In [32]:
X_test=df_valid.drop('Price in $',axis=1)
y_test=df_valid['Price in $']
X_test.shape

(754, 9)

In [33]:
X=df_rank.drop(['Price in $'],axis=1)
y=df_rank['Price in $']
X_train2, X_test2, y_train2, y_test2 = train_test_split( X, y, test_size=0.33, random_state=42)

In [34]:
model = Pipeline([('full_pipeline', full_pipeline),('clf', XGBRegressor())])
model.fit(X_train2,y_train2)
y_pred2=model.predict(X_test2)
mape_rf=mean_absolute_percentage_error(y_test2, y_pred2)
mse_rf=mean_squared_error(y_test2, y_pred2)
mae_rf=mean_absolute_error(y_test2, y_pred2)
print('mape_rf',mape_rf)
print('mse_rf',mse_rf)
print('mae_rf',mae_rf)

mape_rf 0.1956503769256128
mse_rf 181387.0132127729
mae_rf 265.01773254228107


In [35]:
model = Pipeline([('full_pipeline', full_pipeline),('clf', RandomForestRegressor())])
model.fit(X_train2,y_train2)
y_pred2=model.predict(X_test2)
mape_rf=mean_absolute_percentage_error(y_test2, y_pred2)
mse_rf=mean_squared_error(y_test2, y_pred2)
mae_rf=mean_absolute_error(y_test2, y_pred2)
print('mape_rf',mape_rf)
print('mse_rf',mse_rf)
print('mae_rf',mae_rf)

mape_rf 0.2150736697897841
mse_rf 260483.36886453742
mae_rf 307.70415526558014


In [36]:
model = Pipeline([('full_pipeline', full_pipeline),('clf', LGBMRegressor())])
model.fit(X_train2,y_train2)
y_pred2=model.predict(X_test2)
mape_rf=mean_absolute_percentage_error(y_test2, y_pred2)
mse_rf=mean_squared_error(y_test2, y_pred2)
mae_rf=mean_absolute_error(y_test2, y_pred2)
print('mape_rf',mape_rf)
print('mse_rf',mse_rf)
print('mae_rf',mae_rf)

mape_rf 0.22041811160219993
mse_rf 286620.9034006702
mae_rf 317.38465047073885


In [37]:
model = Pipeline([('full_pipeline', full_pipeline),('clf', CatBoostRegressor())])
model.fit(X_train2,y_train2)
y_pred2=model.predict(X_test2)
mape_rf=mean_absolute_percentage_error(y_test2, y_pred2)
mse_rf=mean_squared_error(y_test2, y_pred2)
mae_rf=mean_absolute_error(y_test2, y_pred2)
print('mape_rf',mape_rf)
print('mse_rf',mse_rf)
print('mae_rf',mae_rf)

Learning rate set to 0.044601
0:	learn: 1113.1851851	total: 1.27ms	remaining: 1.27s
1:	learn: 1091.1518744	total: 2.56ms	remaining: 1.28s
2:	learn: 1068.7082161	total: 3.79ms	remaining: 1.26s
3:	learn: 1047.1274925	total: 5.05ms	remaining: 1.26s
4:	learn: 1025.5874919	total: 6.26ms	remaining: 1.25s
5:	learn: 1005.7797105	total: 7.47ms	remaining: 1.24s
6:	learn: 986.8570738	total: 8.91ms	remaining: 1.26s
7:	learn: 968.5816823	total: 10.3ms	remaining: 1.28s
8:	learn: 951.3237997	total: 11.6ms	remaining: 1.28s
9:	learn: 934.2704189	total: 13.1ms	remaining: 1.29s
10:	learn: 920.3744755	total: 14.1ms	remaining: 1.27s
11:	learn: 908.2992884	total: 15.4ms	remaining: 1.26s
12:	learn: 894.6262599	total: 16.7ms	remaining: 1.27s
13:	learn: 882.6723936	total: 18.3ms	remaining: 1.29s
14:	learn: 869.6098055	total: 19.6ms	remaining: 1.28s
15:	learn: 856.3592291	total: 21ms	remaining: 1.29s
16:	learn: 843.9003451	total: 22.4ms	remaining: 1.29s
17:	learn: 833.2835055	total: 23.6ms	remaining: 1.29s
18:	

248:	learn: 402.3593869	total: 292ms	remaining: 882ms
249:	learn: 401.8150999	total: 294ms	remaining: 881ms
250:	learn: 401.5090148	total: 295ms	remaining: 880ms
251:	learn: 400.9502446	total: 296ms	remaining: 878ms
252:	learn: 400.0666985	total: 297ms	remaining: 877ms
253:	learn: 399.5915844	total: 298ms	remaining: 875ms
254:	learn: 399.4084294	total: 299ms	remaining: 874ms
255:	learn: 398.5689647	total: 300ms	remaining: 872ms
256:	learn: 398.0130401	total: 301ms	remaining: 871ms
257:	learn: 397.8140888	total: 303ms	remaining: 870ms
258:	learn: 397.2167458	total: 304ms	remaining: 869ms
259:	learn: 396.8352633	total: 305ms	remaining: 868ms
260:	learn: 396.0873708	total: 306ms	remaining: 867ms
261:	learn: 395.6037496	total: 307ms	remaining: 866ms
262:	learn: 394.8283248	total: 308ms	remaining: 864ms
263:	learn: 394.4771087	total: 310ms	remaining: 863ms
264:	learn: 394.1894715	total: 311ms	remaining: 862ms
265:	learn: 393.3866558	total: 312ms	remaining: 862ms
266:	learn: 392.5373651	tota

402:	learn: 340.5050731	total: 467ms	remaining: 692ms
403:	learn: 340.0561414	total: 469ms	remaining: 691ms
404:	learn: 339.8114038	total: 470ms	remaining: 690ms
405:	learn: 339.7811024	total: 471ms	remaining: 688ms
406:	learn: 339.4031498	total: 471ms	remaining: 687ms
407:	learn: 339.1803619	total: 472ms	remaining: 685ms
408:	learn: 338.9347425	total: 473ms	remaining: 684ms
409:	learn: 338.5678695	total: 474ms	remaining: 682ms
410:	learn: 338.4066687	total: 475ms	remaining: 681ms
411:	learn: 338.1703475	total: 476ms	remaining: 680ms
412:	learn: 337.8734421	total: 477ms	remaining: 679ms
413:	learn: 337.6686001	total: 478ms	remaining: 677ms
414:	learn: 337.5840759	total: 479ms	remaining: 676ms
415:	learn: 337.3148278	total: 481ms	remaining: 675ms
416:	learn: 336.8939011	total: 481ms	remaining: 673ms
417:	learn: 336.8647194	total: 482ms	remaining: 672ms
418:	learn: 336.5871278	total: 483ms	remaining: 670ms
419:	learn: 336.3294846	total: 485ms	remaining: 669ms
420:	learn: 336.0669723	tota

590:	learn: 304.2193534	total: 693ms	remaining: 479ms
591:	learn: 303.9056777	total: 694ms	remaining: 478ms
592:	learn: 303.8963434	total: 695ms	remaining: 477ms
593:	learn: 303.6670264	total: 697ms	remaining: 476ms
594:	learn: 303.5150683	total: 698ms	remaining: 475ms
595:	learn: 303.2937166	total: 700ms	remaining: 474ms
596:	learn: 303.1519489	total: 701ms	remaining: 473ms
597:	learn: 302.7016111	total: 702ms	remaining: 472ms
598:	learn: 302.5393680	total: 704ms	remaining: 471ms
599:	learn: 302.5306343	total: 705ms	remaining: 470ms
600:	learn: 302.1874552	total: 706ms	remaining: 469ms
601:	learn: 302.0275475	total: 708ms	remaining: 468ms
602:	learn: 301.8646324	total: 709ms	remaining: 467ms
603:	learn: 301.7706518	total: 710ms	remaining: 466ms
604:	learn: 301.7590531	total: 711ms	remaining: 464ms
605:	learn: 301.5604123	total: 713ms	remaining: 464ms
606:	learn: 301.3479617	total: 714ms	remaining: 462ms
607:	learn: 301.2317907	total: 715ms	remaining: 461ms
608:	learn: 300.9253692	tota

845:	learn: 269.8712196	total: 992ms	remaining: 181ms
846:	learn: 269.7588267	total: 993ms	remaining: 179ms
847:	learn: 269.6551220	total: 995ms	remaining: 178ms
848:	learn: 269.5823306	total: 996ms	remaining: 177ms
849:	learn: 269.3952808	total: 997ms	remaining: 176ms
850:	learn: 269.3654627	total: 998ms	remaining: 175ms
851:	learn: 269.1981785	total: 999ms	remaining: 173ms
852:	learn: 269.0347383	total: 1000ms	remaining: 172ms
853:	learn: 269.0318167	total: 1s	remaining: 171ms
854:	learn: 268.9058083	total: 1s	remaining: 170ms
855:	learn: 268.8188016	total: 1s	remaining: 169ms
856:	learn: 268.5926615	total: 1s	remaining: 167ms
857:	learn: 268.5329106	total: 1s	remaining: 166ms
858:	learn: 268.4816612	total: 1s	remaining: 165ms
859:	learn: 268.4208765	total: 1.01s	remaining: 164ms
860:	learn: 268.2875068	total: 1.01s	remaining: 163ms
861:	learn: 268.1102835	total: 1.01s	remaining: 162ms
862:	learn: 268.0271515	total: 1.01s	remaining: 160ms
863:	learn: 267.9876371	total: 1.01s	remainin

mape_rf 0.19727591466257244
mse_rf 180792.13108543117
mae_rf 266.4087349751067


In [38]:
df_rank.head()

,CPU,RAM,Storage,GPU,ScreenSize,ScreenType,TouchType,Form,Region,Price in $
0,19,10,10,1,15,UHD,Non-Touch,Laptop,CN,2803.20
1,19,10,10,1,14,FHD,Non-Touch,Laptop,CN,719.85
2,19,10,10,1,15,FHD,Touch,Laptop,BR,1584.00
3,19,10,10,1,13,FHD,Touch,Laptop,BR,1872.00
4,19,10,10,1,15,FHD,Non-Touch,Laptop,BR,1782.00


In [39]:
df_valid.head()

,CPU,RAM,Storage,GPU,ScreenSize,ScreenType,TouchType,Form,Region,Price in $
0,2,4,17,1,14,HD+,Non-Touch,Laptop,BR,359.82
1,8,7,6,1,13,FHD,Non-Touch,Laptop,US,849.99
2,9,10,11,1,14,FHD,Non-Touch,Laptop,BR,1781.82
3,9,7,11,1,13,QHD+,Non-Touch,Laptop,US,599.00
4,13,7,30,1,DT,DT,DT,Desktop,US,367.99


In [40]:
X_train=df_rank.drop('Price in $',axis=1)
y_train=df_rank['Price in $']
X_test=df_valid.drop('Price in $',axis=1)
y_test=df_valid['Price in $']


In [41]:
model = Pipeline([('full_pipeline', full_pipeline),('clf', CatBoostRegressor())])
model.fit(X_train,y_train)
y_pred1=model.predict(X_test)
mape_rf=mean_absolute_percentage_error(y_test, y_pred1)
mse_rf=mean_squared_error(y_test, y_pred1)
mae_rf=mean_absolute_error(y_test, y_pred1)
print('mape_rf',mape_rf)
print('mse_rf',mse_rf)
print('mae_rf',mae_rf)

Learning rate set to 0.047898
0:	learn: 1144.0417020	total: 1.12ms	remaining: 1.12s
1:	learn: 1117.9884073	total: 2.27ms	remaining: 1.13s
2:	learn: 1091.7698981	total: 3.35ms	remaining: 1.11s
3:	learn: 1067.4618979	total: 4.46ms	remaining: 1.11s
4:	learn: 1042.6054213	total: 5.55ms	remaining: 1.1s
5:	learn: 1019.5114922	total: 6.66ms	remaining: 1.1s
6:	learn: 997.4805638	total: 7.95ms	remaining: 1.13s
7:	learn: 978.6491239	total: 9.06ms	remaining: 1.12s
8:	learn: 959.6172929	total: 10.2ms	remaining: 1.12s
9:	learn: 941.0473625	total: 11.2ms	remaining: 1.11s
10:	learn: 927.3898244	total: 12.3ms	remaining: 1.11s
11:	learn: 913.2915066	total: 13.4ms	remaining: 1.1s
12:	learn: 897.0843629	total: 14.5ms	remaining: 1.1s
13:	learn: 883.6902262	total: 15.8ms	remaining: 1.11s
14:	learn: 870.8303918	total: 16.8ms	remaining: 1.11s
15:	learn: 858.7776132	total: 18ms	remaining: 1.11s
16:	learn: 844.9015465	total: 19.2ms	remaining: 1.11s
17:	learn: 834.2728140	total: 20.4ms	remaining: 1.11s
18:	lear

229:	learn: 406.9370252	total: 270ms	remaining: 905ms
230:	learn: 406.1387938	total: 271ms	remaining: 904ms
231:	learn: 405.3664707	total: 273ms	remaining: 902ms
232:	learn: 404.9244945	total: 274ms	remaining: 901ms
233:	learn: 404.5442164	total: 275ms	remaining: 899ms
234:	learn: 404.2206342	total: 276ms	remaining: 897ms
235:	learn: 403.5540098	total: 277ms	remaining: 896ms
236:	learn: 403.0810238	total: 278ms	remaining: 894ms
237:	learn: 402.4706568	total: 279ms	remaining: 893ms
238:	learn: 401.9917215	total: 280ms	remaining: 891ms
239:	learn: 401.2233033	total: 281ms	remaining: 890ms
240:	learn: 400.7329773	total: 282ms	remaining: 890ms
241:	learn: 400.3094778	total: 284ms	remaining: 889ms
242:	learn: 399.7606517	total: 285ms	remaining: 887ms
243:	learn: 399.1628833	total: 286ms	remaining: 886ms
244:	learn: 398.3162044	total: 287ms	remaining: 885ms
245:	learn: 397.8964233	total: 288ms	remaining: 884ms
246:	learn: 397.1839751	total: 290ms	remaining: 883ms
247:	learn: 396.6023646	tota

448:	learn: 336.0596389	total: 530ms	remaining: 651ms
449:	learn: 335.8256714	total: 531ms	remaining: 650ms
450:	learn: 335.3565888	total: 533ms	remaining: 648ms
451:	learn: 335.3217234	total: 534ms	remaining: 647ms
452:	learn: 335.1193277	total: 535ms	remaining: 646ms
453:	learn: 334.9747965	total: 536ms	remaining: 644ms
454:	learn: 334.5762817	total: 537ms	remaining: 643ms
455:	learn: 334.3183840	total: 538ms	remaining: 641ms
456:	learn: 334.1300733	total: 539ms	remaining: 640ms
457:	learn: 333.9768122	total: 540ms	remaining: 639ms
458:	learn: 333.5546634	total: 541ms	remaining: 638ms
459:	learn: 333.3156955	total: 542ms	remaining: 636ms
460:	learn: 333.2686195	total: 543ms	remaining: 635ms
461:	learn: 332.9922730	total: 544ms	remaining: 634ms
462:	learn: 332.7377729	total: 545ms	remaining: 633ms
463:	learn: 332.5438321	total: 546ms	remaining: 631ms
464:	learn: 332.4050254	total: 548ms	remaining: 630ms
465:	learn: 332.2356808	total: 549ms	remaining: 629ms
466:	learn: 331.9843614	tota

603:	learn: 310.2060042	total: 708ms	remaining: 464ms
604:	learn: 310.1155168	total: 709ms	remaining: 463ms
605:	learn: 309.8653253	total: 710ms	remaining: 462ms
606:	learn: 309.7304369	total: 711ms	remaining: 460ms
607:	learn: 309.5766031	total: 712ms	remaining: 459ms
608:	learn: 309.5309533	total: 713ms	remaining: 458ms
609:	learn: 309.3984069	total: 714ms	remaining: 457ms
610:	learn: 309.3284891	total: 716ms	remaining: 456ms
611:	learn: 309.1288573	total: 717ms	remaining: 454ms
612:	learn: 309.1130963	total: 718ms	remaining: 453ms
613:	learn: 308.9163011	total: 719ms	remaining: 452ms
614:	learn: 308.7729837	total: 720ms	remaining: 451ms
615:	learn: 308.6106136	total: 721ms	remaining: 449ms
616:	learn: 308.4841513	total: 722ms	remaining: 448ms
617:	learn: 308.3751266	total: 723ms	remaining: 447ms
618:	learn: 308.1024928	total: 725ms	remaining: 446ms
619:	learn: 308.0928881	total: 726ms	remaining: 445ms
620:	learn: 307.9751088	total: 727ms	remaining: 444ms
621:	learn: 307.8446405	tota

761:	learn: 290.4741514	total: 887ms	remaining: 277ms
762:	learn: 290.3787525	total: 888ms	remaining: 276ms
763:	learn: 290.2121706	total: 889ms	remaining: 275ms
764:	learn: 290.0695790	total: 890ms	remaining: 273ms
765:	learn: 289.9667483	total: 891ms	remaining: 272ms
766:	learn: 289.8758151	total: 892ms	remaining: 271ms
767:	learn: 289.8170528	total: 893ms	remaining: 270ms
768:	learn: 289.5535544	total: 894ms	remaining: 269ms
769:	learn: 289.4809735	total: 895ms	remaining: 267ms
770:	learn: 289.4753072	total: 896ms	remaining: 266ms
771:	learn: 289.3521866	total: 898ms	remaining: 265ms
772:	learn: 289.2766114	total: 899ms	remaining: 264ms
773:	learn: 289.1919149	total: 900ms	remaining: 263ms
774:	learn: 289.0821425	total: 901ms	remaining: 262ms
775:	learn: 288.8797845	total: 903ms	remaining: 261ms
776:	learn: 288.7530337	total: 904ms	remaining: 259ms
777:	learn: 288.6732663	total: 905ms	remaining: 258ms
778:	learn: 288.5810134	total: 906ms	remaining: 257ms
779:	learn: 288.4689136	tota

917:	learn: 275.4237469	total: 1.06s	remaining: 95.2ms
918:	learn: 275.4213051	total: 1.07s	remaining: 94ms
919:	learn: 275.3630842	total: 1.07s	remaining: 92.9ms
920:	learn: 275.3080764	total: 1.07s	remaining: 91.7ms
921:	learn: 275.2180580	total: 1.07s	remaining: 90.6ms
922:	learn: 275.2145881	total: 1.07s	remaining: 89.4ms
923:	learn: 275.1823435	total: 1.07s	remaining: 88.3ms
924:	learn: 275.1098888	total: 1.07s	remaining: 87.1ms
925:	learn: 274.9974944	total: 1.07s	remaining: 86ms
926:	learn: 274.9907919	total: 1.08s	remaining: 84.8ms
927:	learn: 274.8314620	total: 1.08s	remaining: 83.6ms
928:	learn: 274.8130727	total: 1.08s	remaining: 82.5ms
929:	learn: 274.7534321	total: 1.08s	remaining: 81.3ms
930:	learn: 274.6042855	total: 1.08s	remaining: 80.2ms
931:	learn: 274.5033186	total: 1.08s	remaining: 79ms
932:	learn: 274.4303856	total: 1.08s	remaining: 77.9ms
933:	learn: 274.4265812	total: 1.08s	remaining: 76.7ms
934:	learn: 274.2993435	total: 1.09s	remaining: 75.5ms
935:	learn: 274.

In [42]:
model = Pipeline([('full_pipeline', full_pipeline),('clf', XGBRegressor())])
model.fit(X_train,y_train)
y_pred2=model.predict(X_test)
mape_rf=mean_absolute_percentage_error(y_test, y_pred2)
mse_rf=mean_squared_error(y_test, y_pred2)
mae_rf=mean_absolute_error(y_test, y_pred2FCV
print('mape_rf',mape_rf)
print('mse_rf',mse_rf)
print('mae_rf',mae_rf)

mape_rf 0.2713383122629517
mse_rf 744911.0291617482
mae_rf 503.2403096947789


In [43]:
model = Pipeline([('full_pipeline', full_pipeline),('clf', LGBMRegressor())])
model.fit(X_train,y_train)
y_pred3=model.predict(X_test)
mape_rf=mean_absolute_percentage_error(y_test, y_pred3)
mse_rf=mean_squared_error(y_test, y_pred3)
mae_rf=mean_absolute_error(y_test, y_pred3)
print('mape_rf',mape_rf)
print('mse_rf',mse_rf)
print('mae_rf',mae_rf)

mape_rf 0.2849965228172574
mse_rf 686253.0250416679
mae_rf 499.8400990446725


In [44]:
y_pred4=(y_pred1+ y_pred2 + y_pred3 )/3
print(mean_absolute_percentage_error(y_test, y_pred4))

0.27257046468447493


In [45]:
y_pred5=(y_pred1+ y_pred2)/2
print(mean_absolute_percentage_error(y_test, y_pred5))

0.27269934954166175


In [46]:
y_pred6=(y_pred2+ y_pred3)/2
print(mean_absolute_percentage_error(y_test, y_pred6))

0.27182297354813034
